In [0]:
# 1 - Installation of the library Bokeh

%pip install bokeh

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
#File's directories
ctFile = ("dbfs:/FileStore/tables/clinicaltrial_2021")
mshFile = ("dbfs:/FileStore/tables/mesh.csv")
pharFile = ("dbfs:/FileStore/tables/pharma.csv")

#File's year
ctYear = ("2021")

#File's delimiters
ctDelimiter = ("|")
mshDelimiter = (",")
pharDelimiter = ('","')

#Collumns from clienttrial file
typeCol = 5
condCol = 7
sponCol = 1
statCol = 2
compCol = 4

#Collumns from pharma file
pcCol = 1

#Collumn's delimiters
condDelimiter = (",")
nullDelimiter = ('')
rootDelimiter = (".")
sponDeliminiter = ('","')
compDelimiter = (" ")
stats = ("Completed")

#Visualisations
vTitle = ("Completed Studies - 2021")


In [0]:
# 1 - Create ctRDD from ctFile
# 2 - Create ctHeader that extracts the header (first line) from the ctRDD
# 3 - Create ctStudies without the header
# 4 - Check the output with the header

ctRDD = sc.textFile(ctFile)

ctHeader = ctRDD.first()

ctStudies = ctRDD.filter(lambda f: f != ctHeader)

ctRDD.take(2)

Out[2]: ['Id|Sponsor|Status|Start|Completion|Type|Submission|Conditions|Interventions',
 'NCT02758028|The University of Hong Kong|Recruiting|Aug 2005|Nov 2021|Interventional|Apr 2016||']

In [0]:
# 5 - Check the resulting output without the header

ctStudies.take(2)

Out[3]: ['NCT02758028|The University of Hong Kong|Recruiting|Aug 2005|Nov 2021|Interventional|Apr 2016||',
 'NCT02751957|Duke University|Completed|Jul 2016|Jul 2020|Interventional|Apr 2016|Autistic Disorder,Autism Spectrum Disorder|']

In [0]:
# 1 - Number of studies in the dataset

print(ctYear , ctStudies.distinct().count())

2021 387261


In [0]:
# 1 - Split the ctStudies RDD using "|" as the delimiter
# 2 - Check the resulting list of element separated

import re
ctSplit = ctStudies.map(lambda line: line.split(ctDelimiter))
ctSplit.take(2)

Out[5]: [['NCT02758028',
  'The University of Hong Kong',
  'Recruiting',
  'Aug 2005',
  'Nov 2021',
  'Interventional',
  'Apr 2016',
  '',
  ''],
 ['NCT02751957',
  'Duke University',
  'Completed',
  'Jul 2016',
  'Jul 2020',
  'Interventional',
  'Apr 2016',
  'Autistic Disorder,Autism Spectrum Disorder',
  '']]

In [0]:
# 1 - Select the "Type" column in the clinicatrial_21 dataset
# 2 - Create a key-value pair with 1 as the value for each key
# 3 - Count the frequency of each key
# 4 - Order the results in a descending manner
# 5 - Check the resulting output with all the types of studies in the dataset along with their frequencies

import re
ctTypes = ctSplit.map(lambda fields: fields[typeCol])\
                .flatMap(lambda s: [(s, 1)])\
                .reduceByKey(lambda v1,v2: v1+v2)\
                .sortBy(lambda a: a[1],False)
ctTypes.collect()

Out[6]: [('Interventional', 301472),
 ('Observational', 77540),
 ('Observational [Patient Registry]', 8180),
 ('Expanded Access', 69)]

In [0]:
# 1 - Select the Conditions column in the clinicaltrial_21 dataset
# 2 - Clean the dataset deleting the empty spaces
# 3 - Split the ctSplit RDD by comma and list the content of the RDD
# 4 - Create a key-value pair with 1 as the value for each key
# 5 - Count the frequency of each key and order the results in a descending manner
# 6 - Check the resulting output with the top 5 conditions with their frequencies

ctCond = ctSplit.map(lambda fields: fields[condCol])\
            .filter(lambda x: x != nullDelimiter)\
            .map(lambda word: word.split(condDelimiter))\
            .flatMap(lambda s: s)\
            .flatMap(lambda v: [(v,1)])\
            .reduceByKey(lambda x,y: x+y)\
            .sortBy(lambda a: a[1],False)\
           
ctCond.take(6)


Out[7]: [('Carcinoma', 13389),
 ('Diabetes Mellitus', 11080),
 ('Neoplasms', 9371),
 ('Breast Neoplasms', 8640),
 ('Syndrome', 8032),
 ('Leukemia', 5904)]

In [0]:
# 1 - Creates a list of a key-value pair RDD with Conditions

ctCondList = ctSplit.map(lambda fields: fields[condCol])\
            .filter(lambda x: x != nullDelimiter)\
            .map(lambda word: word.split(condDelimiter))\
            .flatMap(lambda s: s)\
            .flatMap(lambda v: [(v,1)])
ctCondList.take(5)

Out[8]: [('Autistic Disorder', 1),
 ('Autism Spectrum Disorder', 1),
 ('Diabetes Mellitus', 1),
 ('Tuberculosis', 1),
 ('Lung Diseases', 1)]

In [0]:
# 1 - Creates an new RDD from the "mshFile" with "mshDelimiter" as delimiter

mshRDD = sc.textFile(mshFile)
smshRDD = mshRDD.map(lambda s: s.split(mshDelimiter))
smshRDD.take(5)

Out[9]: [['term', 'tree'],
 ['Calcimycin', 'D03.633.100.221.173'],
 ['A-23187', 'D03.633.100.221.173'],
 ['Temefos', 'D02.705.400.625.800'],
 ['Temefos', 'D02.705.539.345.800']]

In [0]:
# 1 - Joins the "ctCondList" with "smshRDD"

joinCondMsh = ctCondList.join(smshRDD)

joinCondMsh.take(5)

Out[10]: [('Autistic Disorder', (1, 'F03.625.164.113.500')),
 ('Autistic Disorder', (1, 'F03.625.164.113.500')),
 ('Autistic Disorder', (1, 'F03.625.164.113.500')),
 ('Autistic Disorder', (1, 'F03.625.164.113.500')),
 ('Autistic Disorder', (1, 'F03.625.164.113.500'))]

In [0]:
# 1 - Finds the top 5 roots most frequent roots

topRoots = joinCondMsh.map(lambda fields: fields[1])\
                .map(lambda fields: fields[1])\
                .map(lambda sp: sp.split(rootDelimiter))\
                .map(lambda f: f[0])\
                .flatMap(lambda v: [(v, 1)])\
                .reduceByKey(lambda x,y: x+y)\
                .sortBy(lambda a: a[1], False)
topRoots.take(5)

Out[11]: [('C04', 143994),
 ('C23', 136079),
 ('C01', 106674),
 ('C14', 94523),
 ('C10', 92310)]

In [0]:
# 1 - Creates a list of a key-value pair RDD with Sponsors

ctSponList = ctSplit.map(lambda fields: fields[sponCol])\
            .map(lambda lw: lw.split(sponDeliminiter))\
            .flatMap(lambda s: s)\
            .flatMap(lambda v: [(v,1)])
            
ctSponList.collect()

Out[12]: [('The University of Hong Kong', 1),
 ('Duke University', 1),
 ('Universidade Federal do Rio de Janeiro', 1),
 ('Istanbul Medeniyet University', 1),
 ('University of Roma La Sapienza', 1),
 ('Consorzio Futuro in Ricerca', 1),
 ('Ankara University', 1),
 ('Ruijin Hospital', 1),
 ('Washington University School of Medicine', 1),
 ('Orphazyme', 1),
 ('Novo Nordisk A/S', 1),
 ('Daniel Alexandre Bottino', 1),
 ('Bulent Ecevit University', 1),
 ('Institut für Pharmakologie und Präventive Medizin', 1),
 ('The Third Xiangya Hospital of Central South University', 1),
 ('Tel Aviv Medical Center', 1),
 ('Medicines for Malaria Venture', 1),
 ('James Cook University, Queensland, Australia', 1),
 ('Soonchunhyang University Hospital', 1),
 ('Member Companies of the Opioid PMR Consortium', 1),
 ('Marmara University', 1),
 ('Cycle Pharmaceuticals Ltd.', 1),
 ('Yonsei University', 1),
 ('Cutera Inc.', 1),
 ('University of Zurich', 1),
 ('Guangdong Association of Clinical Trials', 1),
 ('Ruijin H

In [0]:
# 1 - Creates a new RDD with the pharma dataset
# 2 - Creates a key-value list for the Parent Company column

import re
pharRDD = sc.textFile(pharFile)
pharmList = pharRDD.map(lambda l: l.split(pharDelimiter))\
            .map(lambda fields: fields[pcCol])\
            .flatMap(lambda v: [(v, 1)])
pharmList.take(10)

Out[13]: [('Parent_Company', 1),
 ('Abbott Laboratories', 1),
 ('AbbVie', 1),
 ('AbbVie', 1),
 ('Abbott Laboratories', 1),
 ('Johnson & Johnson', 1),
 ('Abbott Laboratories', 1),
 ('Abbott Laboratories', 1),
 ('Johnson & Johnson', 1),
 ('Johnson & Johnson', 1)]

In [0]:
topSpon = ctSponList.leftOuterJoin(pharmList)\
                .subtractByKey(pharmList)\
                .map(lambda f: f[0])\
                .flatMap(lambda v: [(v, 1)])\
                .reduceByKey(lambda x,y: x+y)\
                .sortBy(lambda a: a[1], False)
                
topSpon.take(10)

Out[14]: [('National Cancer Institute (NCI)', 3218),
 ('M.D. Anderson Cancer Center', 2414),
 ('Assistance Publique - Hôpitaux de Paris', 2369),
 ('Mayo Clinic', 2300),
 ('Merck Sharp & Dohme Corp.', 2243),
 ('Assiut University', 2154),
 ('Novartis Pharmaceuticals', 2088),
 ('Massachusetts General Hospital', 1971),
 ('Cairo University', 1928),
 ('Hoffmann-La Roche', 1828)]

In [0]:
import calendar

# 1 - Dictionary for months
months = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}

compStudies = ctSplit.map(lambda fields: (fields[statCol], fields[compCol]))\
                .filter(lambda a: a != ctHeader)\
                .map(lambda b: (b[0], b[1]))\
                .filter(lambda c: stats in c[0])\
                .map(lambda d: d[1])\
                .map(lambda e: e.split(compDelimiter))\
                .filter(lambda f: f[0] != nullDelimiter)\
                .filter(lambda g: g[1] == ctYear)\
                .map(lambda h: h[0])\
                .flatMap(lambda v: [(v, 1)])\
                .reduceByKey(lambda x,y: x+y)\
                .sortBy(keyfunc=lambda x: months.get(x[0]))
                
compStudies.collect()

Out[15]: [('Jan', 1131),
 ('Feb', 934),
 ('Mar', 1227),
 ('Apr', 967),
 ('May', 984),
 ('Jun', 1094),
 ('Jul', 819),
 ('Aug', 700),
 ('Sep', 528),
 ('Oct', 187)]

In [0]:
from bokeh.io import output_file, show
from bokeh.transform import dodge
from bokeh.plotting import figure
from bokeh.embed import components, file_html
from bokeh.resources import CDN
from bokeh.models import ColumnDataSource

# 1 - Chooses the graph type
output_file("bars.html")

# 2 - Create a Dataframe from an RDD
compSDF = compStudies.toDF()
csDF = compSDF.withColumnRenamed('_1','Month').withColumnRenamed('_2','Count')

# 3 - Convert the Dataframe to Pandas
dataP = csDF.toPandas()
source = ColumnDataSource(dataP)

# 4 - Assign values from column month to the graph
months = source.data['Month'].tolist()
p = figure(x_range= months)

# 5 - Graph configutarion
p = figure(x_range=months, height=250, title=vTitle,
           toolbar_location=None, tools="")

p.vbar(x='Month', top='Count', source = source, width=0.9)


p.xgrid.grid_line_color = None
p.y_range.start = 0

html =file_html(p, CDN, "my plot1")

displayHTML(html)

<!DOCTYPE html>
 
 
 
 
 
 my plot1